# Quick and dirty reference sheet for Tim

## Training JSAEs
Run `train.py` and change any arguments you need. That automatically uploads the trained model to Wandb.

## Using pre-trained JSAEs

You can download the sae_pair like this. When you're using it, you basically always pass in a boolean argument which specifies which SAE to use (False = pre-MLP SAE; True = post-MLP SAE).

In [1]:
import sys
sys.path.append('..')
from jacobian_saes.utils import load_pretrained, run_sandwich

# Load the pretrained model
wandb_path = "lucyfarnik/jsaes_pythia70m1/sae_pythia-70m-deduped_blocks.3.ln2.hook_normalized_16384:v0"
sae_pair, model, mlp_with_grads, layer = load_pretrained(wandb_path)

_, cache = model.run_with_cache("Hello world!", stop_at_layer=layer+1,
                                names_filter=[sae_pair.cfg.hook_name])
acts = cache[sae_pair.cfg.hook_name]

# Encoding and decoding
encoded, topk_indices = sae_pair.encode(acts, False, return_topk_indices=True)
reconstructed = sae_pair.decode(encoded, False)

# Run the sandwich
jacobian, acts_dict = run_sandwich(sae_pair, mlp_with_grads, acts)

/Users/tz20913/Library/Mobile Documents/com~apple~CloudDocs/Desktop/Research/Jacobian SAEs/notebooks/../jacobian_saes/sae_pair.py:146: UserWarning: 
This SAE has non-empty model_from_pretrained_kwargs. 
For optimal performance, load the model like so:
model = HookedSAETransformer.from_pretrained_no_processing(..., **cfg.model_from_pretrained_kwargs)
  warnings.warn(


Loaded pretrained model pythia-70m-deduped into HookedTransformer


## Important: the first SAE's activations should not be causally upstream of the second SAE's activations

The code below is correct

In [2]:
sae_acts1 = sae_pair.encode(acts, False)
act_reconstr = sae_pair.decode(sae_acts1, False)
mlp_out, _ = mlp_with_grads(acts) #! important bit: we're passing in the original activations
sae_acts2 = sae_pair.encode(mlp_out, True)
mlp_out_reconstr = sae_pair.decode(sae_acts2, True)

The code below is wrong

In [3]:
sae_acts1 = sae_pair.encode(acts, False)
act_reconstr = sae_pair.decode(sae_acts1, False)
mlp_out, _ = mlp_with_grads(act_reconstr) #! important bit: we're passing in the reconstructed activations, which isn't what we trained with
raise ValueError("You passed in the wrong thing")
sae_acts2 = sae_pair.encode(mlp_out, True)
mlp_out_reconstr = sae_pair.decode(sae_acts2, True)

ValueError: You passed in the wrong thing